In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from shutil import copy2
import psycopg2

In [ ]:
def date_changer(df: pd.DataFrame) -> pd.Series:
    """Converts the 'Date' column of a DataFrame to datetime format.

    Args:
        df (pandas.DataFrame): The DataFrame containing the 'Date' column. The 'Date' column
            should be in a format parsable by pandas.to_datetime().

    Returns:
        pandas.Series: The 'Date' column converted to datetime format.
    """

    df['Date'] = pd.to_datetime(df['Date'])

    return df['Date']

In [ ]:
# Define the base folder containing the data folders
base_folder = '../data'

# Loop through each folder within the base folder
for folder in os.listdir(base_folder):

    # Construct the full path to the current folder
    folder_path = os.path.join(base_folder, folder)

    # Check if a specific file named "Chart data.csv" exists in the current folder
    if 'Chart data.csv' in os.listdir(folder_path):
        # Read the "Chart data.csv" file into a pandas DataFrame
        df = pd.read_csv(f'{folder_path}/Chart data.csv')

        # Convert the 'Date' column to datetime format using a separate function (assuming date_changer is defined elsewhere)
        df['Date'] = date_changer(df)

        # Save the modified DataFrame to a new CSV file in the database folder with folder name
        df.to_csv(f'../database_file/{folder}.csv', index=False)

    # Handle folders with only one file (assuming the file is named "Totals.csv")
    elif len(os.listdir(folder_path)) == 1:
        # Get the full path to the single file in the current folder
        source_file = os.path.join(folder_path, os.listdir(folder_path)[0])

        # Set the destination path to the database folder
        destination_path = '../database_file'

        # Construct the initial filename for the copied file (assuming same name as source)
        source_name = os.path.join('../database_file', os.listdir(folder_path)[0])

        # Construct the final filename with the folder name as the CSV suffix
        destination_name = os.path.join('../database_file', f'{folder}.csv')

        # Copy the single file from source to destination folder
        copy2(source_file, destination_path)

        # Rename the copied file to have the folder name as the CSV suffix
        os.rename(source_name, destination_name)

    # Handle folders with more than one file (assuming a file named "Totals.csv" exists)
    else:
        # Set the source file path to "Totals.csv" within the current folder
        source_file = os.path.join(folder_path, 'Totals.csv')

        # Set the destination path to the database folder
        destination_path = '../database_file'

        # Construct the initial filename for the copied file (assuming same name as source)
        source_name = os.path.join('../database_file', 'Totals.csv')

        # Construct the final filename with the folder name as the CSV suffix
        destination_name = os.path.join('../database_file', f'{folder}.csv')

        # Copy the "Totals.csv" file from source to destination folder
        copy2(source_file, destination_path)

        # Rename the copied file to have the folder name as the CSV suffix
        os.rename(source_name, destination_name)


In [ ]:
from typing import final

# Load environment variables from a `.env` file for secure database credentials
load_dotenv('../.env')

# Establish database connection details 
HOST: final = os.getenv('HOST')
PORT: final = os.getenv('PORT')
USERNAME: final = os.getenv('USERNAME')
PASSWORD: final = os.getenv('PASSWORD')
DATABASE: final = os.getenv('DATABASE')


def import_csv_to_table(csv_file: str, table_name: str) -> None:
    """
    Imports data from a CSV file into a specified table in a PostgreSQL database.

    Args:
        csv_file (str): Path to the CSV file containing the data to import.
        table_name (str): Name of the target table in the database.

    Raises:
        Exception: Any exception encountered during the import process.
    """

    try:
        # Connect to the PostgreSQL database 
        conn = psycopg2.connect(host=HOST, port=PORT, user=USERNAME, password=PASSWORD, database=DATABASE)
        cur = conn.cursor()

        # Open the CSV file in read mode and skip the header row 
        with open(csv_file, 'r') as f:
            next(f)

            # Efficiently import data from CSV into the table 
            cur.copy_from(f, table_name, sep=';', null='')

        # Commit changes to the database 
        conn.commit()

    except (Exception, psycopg2.Error) as error:
        # Log or handle import errors gracefully
        print(f'Error importing data fromm{csv_file}: {error}')



In [ ]:
csv_folder = '../database_file'
for filename in os.listdire(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)
        table_name = filename[:-4]
        import_csv_to_table(csv_file,table_name)